# Dendrite Segmentation with Suite2p

This notebook configures suite2p for detecting **dendrites and other elongated neuronal structures** (boutons, axons) rather than cell bodies.

Key changes from default soma detection:
- `connected = False` — allows disconnected ROI components (dendrites fragment easily)
- `soma_crop = False` — preserves full elongated morphology for classification
- `threshold_scaling` lowered — dendrites are dimmer than somata
- `spatial_scale` increased — matches wider spatial extent of dendrites
- `max_overlap` increased — dendrites overlap more than cell bodies

In [1]:
!uv pip install ""../../LBM-Suite2p-Python""
!uv pip install ""../../mbo_utilities[all]""

Using Python 3.12.9 environment at: C:\Users\RBO\repos\mbo_utilities\.venv
Resolved 158 packages in 66ms
   Building lbm-suite2p-python @ file:///C:/Users/RBO/repos/LBM-Suite2p-Python
      Built lbm-suite2p-python @ file:///C:/Users/RBO/repos/LBM-Suite2p-Python
Prepared 1 package in 912ms
Uninstalled 1 package in 3ms
Installed 1 package in 9ms
 ~ lbm-suite2p-python==2.5.3 (from file:///C:/Users/RBO/repos/LBM-Suite2p-Python)
Using Python 3.12.9 environment at: C:\Users\RBO\repos\mbo_utilities\.venv
Resolved 279 packages in 220ms
   Building mbo-utilities @ file:///C:/Users/RBO/repos/mbo_utilities
      Built mbo-utilities @ file:///C:/Users/RBO/repos/mbo_utilities
Prepared 1 package in 1.54s
Uninstalled 1 package in 17ms
Installed 1 package in 28ms
 ~ mbo-utilities==2.5.2 (from file:///C:/Users/RBO/repos/mbo_utilities)


In [2]:
import numpy as np
from pathlib import Path
import mbo_utilities as mbo
import lbm_suite2p_python as lsp

## Configure ops for dendrite detection

Importing suite2p packages...


In [14]:
ops = lsp.default_ops()

ops["anatomical_only"] = 0
ops["connected"] = False        # allow disconnected ROI components
ops["soma_crop"] = False        # keep full dendrite shape for classification

ops["spatial_scale"] = 1  # lower threshold to catch dimmer dendrites
ops["threshold_scaling"] = 0.5  # lower threshold to catch dimmer dendrites
ops["max_overlap"] = 1.0      # more iterations to find elongated ROIs
ops["sparse_mode"] = True       # sparse mode works well for dendrites
ops["allow_overlap"] = True     # assign shared pixels to both ROIs

Importing suite2p packages...


## Run pipeline

In [15]:
data_path = r"E:\datasets\hireslbm\2026-02-10"
data = mbo.imread(data_path)
data.shape

Counting frames:   0%|          | 0/6 [00:00<?, ?it/s]

(10906, 28, 440, 438)

In [16]:
lsp.pipeline(
    data,
    Path(data_path).joinpath("cpsam"),
    ops=ops,
    planes=[1],
    overwrite=True
)  # uncomment to run

Delegating to run_volume (4D input detected)...
Processing 1 planes in volume (Total planes: 28)
Output: E:\datasets\hireslbm\2026-02-10\cpsam

--- Volume Step: Plane 1 ---
Importing suite2p packages...
Writing binary to E:\datasets\hireslbm\2026-02-10\cpsam\zplane01_tp00001-10906...
NOTE: applying default C:\Users\RBO\.suite2p\classifiers\classifier_user.npy
----------- REGISTRATION
Reference frame, 7.24 sec.
Registered 500/10906 in 7.38s
Registered 1000/10906 in 14.66s
Registered 1500/10906 in 21.79s
Registered 2000/10906 in 28.80s
Registered 2500/10906 in 35.78s
Registered 3000/10906 in 42.77s
Registered 3500/10906 in 49.85s
Registered 4000/10906 in 57.05s
Registered 4500/10906 in 64.08s
Registered 5000/10906 in 71.39s
Registered 5500/10906 in 78.61s
Registered 6000/10906 in 85.82s
Registered 6500/10906 in 93.08s
Registered 7000/10906 in 100.33s
Registered 7500/10906 in 108.10s
Registered 8000/10906 in 115.87s
Registered 8500/10906 in 122.61s
Registered 9000/10906 in 130.09s
Registe

[WindowsPath('E:/datasets/hireslbm/2026-02-10/cpsam/zplane01_tp00001-10906/ops.npy')]

## Load and inspect results

In [1]:
results_dir = Path(save_path) / "suite2p" / "plane0"

stat = np.load(results_dir / "stat.npy", allow_pickle=True)
iscell = np.load(results_dir / "iscell.npy")
ops_out = np.load(results_dir / "ops.npy", allow_pickle=True).item()

n_total = len(stat)
n_cells = int(iscell[:, 0].sum())
print(f"detected {n_total} ROIs, {n_cells} classified as cells")

NameError: name 'Path' is not defined

## Visualize dendrite ROIs

In [ ]:
import matplotlib.pyplot as plt

Ly, Lx = ops_out["Ly"], ops_out["Lx"]
img = np.zeros((Ly, Lx), dtype="float32")

# overlay all cell ROIs
for i, s in enumerate(stat):
    if iscell[i, 0]:
        img[s["ypix"], s["xpix"]] += s["lam"]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(ops_out["meanImg"], cmap="gray")
axes[0].set_title("mean image")
axes[0].axis("off")

axes[1].imshow(img, cmap="hot")
axes[1].set_title(f"dendrite ROIs (n={n_cells})")
axes[1].axis("off")

plt.tight_layout()
plt.show()